In [1]:
# !pip install numpy
# !pip install pandas
# !pip install matplotlib
# !pip install openpyxl xlrd pyarrow fastparquet h5py
# !pip install scipy statsmodels
# !pip install scikit-learn

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import glob
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# DataBases preparation

In [3]:
df_VIX = pd.read_csv("VIX.csv")
df_VIX['vix_level'] = df_VIX['VIXCLS']
df_VIX['vix_diff'] = df_VIX['vix_level'].diff()
df_VIX['vi(x_change'] = df_VIX['vix_level'].pct_change()

df_FX = pd.read_csv("USA_Exchange_Rate.csv")
print(df_VIX.head())

  observation_date  VIXCLS  vix_level  vix_diff  vi(x_change
0       1990-01-02   17.24      17.24       NaN          NaN
1       1990-01-03   18.19      18.19      0.95     0.055104
2       1990-01-04   19.22      19.22      1.03     0.056625
3       1990-01-05   20.11      20.11      0.89     0.046306
4       1990-01-08   20.26      20.26      0.15     0.007459


/tmp/ipython-input-2318182593.py:4: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_VIX['vi(x_change'] = df_VIX['vix_level'].pct_change()


VIX plus prope

In [4]:
df_new = pd.DataFrame()
df_new['date'] = pd.to_datetime(df_VIX['observation_date'])
df_new = df_new.drop_duplicates(subset='date').sort_values('date').reset_index(drop=True)
df_new = df_new.set_index('date')
df_new['vix'] = pd.NA
df_new['vix'] = df_VIX.set_index(pd.to_datetime(df_VIX['observation_date']))['vix_level']
df_new['vix'] = df_new['vix'].ffill()
df_new

,vix
date,
1990-01-02,17.24
1990-01-03,18.19
1990-01-04,19.22
1990-01-05,20.11
1990-01-08,20.26
...,...
2026-01-23,16.09
2026-01-26,16.15
2026-01-27,16.35


Data base de taux d'échange

In [5]:
df_FX = pd.read_csv("USA_Exchange_Rate.csv")
df_FX['Dolar to Euro'] = df_FX['DEXUSEU']
df_FX['date'] = pd.to_datetime(df_FX['observation_date'])
df_FX = df_FX.sort_values('date')
df_new.index = pd.to_datetime(df_new.index).normalize()

df_FX['observation_date'] = pd.to_datetime(df_FX['observation_date']).dt.normalize()
df_FX = df_FX.set_index('observation_date')

df_new['fx'] = df_FX['DEXUSEU'].reindex(df_new.index)
df_new['fx'].first_valid_index(), df_new['fx'].last_valid_index()
df_new['fx'].notna().sum()
df_new

df_clean = df_new.copy()
df_clean = df_clean[df_clean['fx'].notna()]



print(df_FX.head())
print(df_clean.head())
print(df_new.iloc[5000])
print(df_new.head())



                  DEXUSEU  Dolar to Euro       date
observation_date                                   
1999-01-04         1.1812         1.1812 1999-01-04
1999-01-05         1.1760         1.1760 1999-01-05
1999-01-06         1.1636         1.1636 1999-01-06
1999-01-07         1.1672         1.1672 1999-01-07
1999-01-08         1.1554         1.1554 1999-01-08
              vix      fx
date                     
1999-01-04  26.17  1.1812
1999-01-05  24.46  1.1760
1999-01-06  23.34  1.1636
1999-01-07  24.37  1.1672
1999-01-08  23.28  1.1554
vix    50.9300
fx      1.2549
Name: 2009-03-03 00:00:00, dtype: float64
              vix  fx
date                 
1990-01-02  17.24 NaN
1990-01-03  18.19 NaN
1990-01-04  19.22 NaN
1990-01-05  20.11 NaN
1990-01-08  20.26 NaN


On esseyera de tout garder sur le dataframe df_clean

In [6]:
df_clean

,vix,fx
date,,
1999-01-04,26.17,1.1812
1999-01-05,24.46,1.1760
1999-01-06,23.34,1.1636
1999-01-07,24.37,1.1672
1999-01-08,23.28,1.1554
...,...,...
2025-12-01,17.24,1.1623
2025-12-02,16.59,1.1596
2025-12-03,16.08,1.1660


L'inflation:

In [7]:
df_inflation_US = pd.read_csv("inflation_us.csv")
df_inflation_US = df_inflation_US.sort_values('observation_date')
df_inflation_US['inflation_mom'] = (df_inflation_US['CPIAUCSL'].pct_change() * 100)

print(df_inflation_US.head(13))
print(df_inflation_US.tail())


   observation_date  CPIAUCSL  inflation_mom
0        1947-01-01     21.48            NaN
1        1947-02-01     21.62       0.651769
2        1947-03-01     22.00       1.757632
3        1947-04-01     22.00       0.000000
4        1947-05-01     21.95      -0.227273
5        1947-06-01     22.08       0.592255
6        1947-07-01     22.23       0.679348
7        1947-08-01     22.40       0.764732
8        1947-09-01     22.84       1.964286
9        1947-10-01     22.91       0.306480
10       1947-11-01     23.06       0.654736
11       1947-12-01     23.41       1.517780
12       1948-01-01     23.68       1.153353
    observation_date  CPIAUCSL  inflation_mom
944       2025-09-01   324.245       0.295090
945       2025-10-01       NaN       0.000000
946       2025-11-01   325.063       0.252278
947       2025-12-01   326.031       0.297788
948       2026-01-01   326.588       0.170843


/tmp/ipython-input-1286773525.py:3: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_inflation_US['inflation_mom'] = (df_inflation_US['CPIAUCSL'].pct_change() * 100)


In [8]:
df_inflation_US = df_inflation_US.dropna()
df_inflation_US.isna().sum()



,0
observation_date,0
CPIAUCSL,0
inflation_mom,0


In [9]:
df_inflation_US['observation_date'] = pd.to_datetime(
    df_inflation_US['observation_date']
).dt.normalize()

df_inflation_US = df_inflation_US.set_index('observation_date')
df_inflation_US = df_inflation_US.sort_index()


In [10]:
inflation_daily = df_inflation_US['inflation_mom'].resample('D').ffill()
inflation_daily.name = 'inflation_us_mom'


In [11]:
inflation_daily

,inflation_us_mom
observation_date,
1947-02-01,0.651769
1947-02-02,0.651769
1947-02-03,0.651769
1947-02-04,0.651769
1947-02-05,0.651769
...,...
2025-12-28,0.297788
2025-12-29,0.297788
2025-12-30,0.297788


In [12]:
df_clean = df_clean.join(inflation_daily, how='left')
df_clean['inflation_us_mom'] = df_clean['inflation_us_mom'].ffill()

In [13]:
df_clean

,vix,fx,inflation_us_mom
date,,,
1999-01-04,26.17,1.1812,0.182482
1999-01-05,24.46,1.1760,0.182482
1999-01-06,23.34,1.1636,0.182482
1999-01-07,24.37,1.1672,0.182482
1999-01-08,23.28,1.1554,0.182482
...,...,...,...
2025-12-01,17.24,1.1623,0.297788
2025-12-02,16.59,1.1596,0.297788
2025-12-03,16.08,1.1660,0.297788


In [14]:
df_inflation_EU = pd.read_csv("inflacion_eu.csv")
df_inflation_EU['DATE'] = pd.to_datetime(df_inflation_EU['DATE'])
df_eu = df_inflation_EU.sort_values('DATE')

df_inflation_EU['inflation_mom'] = (
    df_inflation_EU['HICP Inflation rate - Total - Index (HICP.M.U2.N.000000.4D0.INX)'].pct_change() * 100)

print(df_inflation_EU.head())

        DATE TIME PERIOD  \
0 1996-01-31     1996Jan   
1 1996-02-29     1996Feb   
2 1996-03-31     1996Mar   
3 1996-04-30     1996Apr   
4 1996-05-31     1996May   

   HICP Inflation rate - Total - Index (HICP.M.U2.N.000000.4D0.INX)  \
0                                              55.12                  
1                                              55.37                  
2                                              55.57                  
3                                              55.66                  
4                                              55.79                  

   inflation_mom  
0            NaN  
1       0.453556  
2       0.361206  
3       0.161958  
4       0.233561  


In [15]:
df_inflation_EU = df_inflation_EU.dropna(subset=['inflation_mom'])


In [16]:
df_inflation_EU['DATE'] = pd.to_datetime(df_inflation_EU['DATE']).dt.normalize()
df_inflation_EU = df_inflation_EU.sort_values('DATE')
df_inflation_EU = df_inflation_EU.set_index('DATE')


In [17]:
infl_eu_daily = df_inflation_EU['inflation_mom'].resample('D').ffill()
infl_eu_daily.name = 'inflation_eu_mom'


In [18]:
infl_eu_daily

,inflation_eu_mom
DATE,
1996-02-29,0.453556
1996-03-01,0.453556
1996-03-02,0.453556
1996-03-03,0.453556
1996-03-04,0.453556
...,...
2026-01-27,0.179194
2026-01-28,0.179194
2026-01-29,0.179194


In [19]:
df_clean = df_clean.join(infl_eu_daily, how='left')
df_clean['inflation_eu_mom'] = df_clean['inflation_eu_mom'].ffill()


In [20]:
df_clean

,vix,fx,inflation_us_mom,inflation_eu_mom
date,,,,
1999-01-04,26.17,1.1812,0.182482,0.052356
1999-01-05,24.46,1.1760,0.182482,0.052356
1999-01-06,23.34,1.1636,0.182482,0.052356
1999-01-07,24.37,1.1672,0.182482,0.052356
1999-01-08,23.28,1.1554,0.182482,0.052356
...,...,...,...,...
2025-12-01,17.24,1.1623,0.297788,-0.287870
2025-12-02,16.59,1.1596,0.297788,-0.287870
2025-12-03,16.08,1.1660,0.297788,-0.287870


Les intêrets:

In [21]:
df_interes_US = pd.read_csv("Interes_USA.csv")
df_interes_US['Interes'] = df_interes_US['Value']
df_interes_US['Date'] = pd.to_datetime(
    df_interes_US['Date'],
    dayfirst=False,   # porque es 07/01/1954
    errors='coerce')

print(df_interes_US.head(20))

         Date  Value  Interes
0  1954-07-01   1.13     1.13
1  1954-07-02   1.25     1.25
2  1954-07-03   1.25     1.25
3  1954-07-04   1.25     1.25
4  1954-07-05   0.88     0.88
5  1954-07-06   0.25     0.25
6  1954-07-07   1.00     1.00
7  1954-07-08   1.25     1.25
8  1954-07-09   1.25     1.25
9  1954-07-10   1.25     1.25
10 1954-07-11   1.25     1.25
11 1954-07-12   1.25     1.25
12 1954-07-13   1.13     1.13
13 1954-07-14   1.13     1.13
14 1954-07-15   0.75     0.75
15 1954-07-16   0.75     0.75
16 1954-07-17   0.75     0.75
17 1954-07-18   0.75     0.75
18 1954-07-19   0.50     0.50
19 1954-07-20   0.25     0.25


In [22]:
df_interes_US['Date'] = pd.to_datetime(df_interes_US['Date'], errors='coerce')
df_interes_US = df_interes_US.dropna(subset=['Date'])
df_interes_US = df_interes_US.sort_values('Date')


In [23]:
df_interes_US = df_interes_US.set_index('Date')
df_interes_US['interest_us'] = df_interes_US['Interes']


In [24]:
df_clean = df_clean.join(df_interes_US['interest_us'], how='left')
df_clean['interest_us'] = df_clean['interest_us'].ffill()


In [25]:
df_clean

,vix,fx,inflation_us_mom,inflation_eu_mom,interest_us
date,,,,,
1999-01-04,26.17,1.1812,0.182482,0.052356,5.04
1999-01-05,24.46,1.1760,0.182482,0.052356,4.54
1999-01-06,23.34,1.1636,0.182482,0.052356,4.23
1999-01-07,24.37,1.1672,0.182482,0.052356,4.49
1999-01-08,23.28,1.1554,0.182482,0.052356,4.74
...,...,...,...,...,...
2025-12-01,17.24,1.1623,0.297788,-0.287870,3.89
2025-12-02,16.59,1.1596,0.297788,-0.287870,3.89
2025-12-03,16.08,1.1660,0.297788,-0.287870,3.89


In [26]:
df_interes_EU= pd.read_csv("interes.csv")
df_interes_EU


,DATE,TIME PERIOD,Long-term interest rate for convergence purposes - Euro (IRS.M.U2.L.L40.CI.0000.EUR.N.Z)
0,1990-12-31,1990Dec,9.31
1,1991-01-31,1991Jan,9.32
2,1991-02-28,1991Feb,8.75
3,1991-03-31,1991Mar,9.98
4,1991-04-30,1991Apr,9.83
...,...,...,...
417,2025-09-30,2025Sep,3.15
418,2025-10-31,2025Oct,3.07
419,2025-11-30,2025Nov,3.09
420,2025-12-31,2025Dec,3.21


In [27]:
df_interes_EU['DATE'] = pd.to_datetime(df_interes_EU['DATE'])
df_interes_EU = df_interes_EU.sort_values('DATE')


In [28]:
df_interes_EU = df_interes_EU.set_index('DATE')


In [29]:
rate_eu_daily = df_interes_EU[
    'Long-term interest rate for convergence purposes - Euro (IRS.M.U2.L.L40.CI.0000.EUR.N.Z)'
].resample('D').ffill()

rate_eu_daily.name = 'rate_eu'


In [30]:
df_clean = df_clean.join(rate_eu_daily, how='left')
df_clean['rate_eu'] = df_clean['rate_eu'].ffill()


In [31]:
df_clean['spread_rates'] = df_clean['interest_us'] - df_clean['rate_eu']


In [32]:
df_clean

,vix,fx,inflation_us_mom,inflation_eu_mom,interest_us,rate_eu,spread_rates
date,,,,,,,
1999-01-04,26.17,1.1812,0.182482,0.052356,5.04,3.96,1.08
1999-01-05,24.46,1.1760,0.182482,0.052356,4.54,3.96,0.58
1999-01-06,23.34,1.1636,0.182482,0.052356,4.23,3.96,0.27
1999-01-07,24.37,1.1672,0.182482,0.052356,4.49,3.96,0.53
1999-01-08,23.28,1.1554,0.182482,0.052356,4.74,3.96,0.78
...,...,...,...,...,...,...,...
2025-12-01,17.24,1.1623,0.297788,-0.287870,3.89,3.09,0.80
2025-12-02,16.59,1.1596,0.297788,-0.287870,3.89,3.09,0.80
2025-12-03,16.08,1.1660,0.297788,-0.287870,3.89,3.09,0.80


In [33]:
df_clean = df_clean.drop(columns=['interest_us', 'rate_eu'])


In [34]:
df_clean

,vix,fx,inflation_us_mom,inflation_eu_mom,spread_rates
date,,,,,
1999-01-04,26.17,1.1812,0.182482,0.052356,1.08
1999-01-05,24.46,1.1760,0.182482,0.052356,0.58
1999-01-06,23.34,1.1636,0.182482,0.052356,0.27
1999-01-07,24.37,1.1672,0.182482,0.052356,0.53
1999-01-08,23.28,1.1554,0.182482,0.052356,0.78
...,...,...,...,...,...
2025-12-01,17.24,1.1623,0.297788,-0.287870,0.80
2025-12-02,16.59,1.1596,0.297788,-0.287870,0.80
2025-12-03,16.08,1.1660,0.297788,-0.287870,0.80


Préparation des features :

In [35]:
df_clean['log_return'] = np.log(df_clean['fx'] / df_clean['fx'].shift(1))
df_clean['ret_lag1'] = df_clean['log_return'].shift(1)
df_clean['ret_lag2'] = df_clean['log_return'].shift(2)
df_clean['vix_change'] = df_clean['vix'].pct_change()


In [36]:
df_clean

,vix,fx,inflation_us_mom,inflation_eu_mom,spread_rates,log_return,ret_lag1,ret_lag2,vix_change
date,,,,,,,,,
1999-01-04,26.17,1.1812,0.182482,0.052356,1.08,NaN,NaN,NaN,NaN
1999-01-05,24.46,1.1760,0.182482,0.052356,0.58,-0.004412,NaN,NaN,-0.065342
1999-01-06,23.34,1.1636,0.182482,0.052356,0.27,-0.010600,-0.004412,NaN,-0.045789
1999-01-07,24.37,1.1672,0.182482,0.052356,0.53,0.003089,-0.010600,-0.004412,0.044130
1999-01-08,23.28,1.1554,0.182482,0.052356,0.78,-0.010161,0.003089,-0.010600,-0.044727
...,...,...,...,...,...,...,...,...,...
2025-12-01,17.24,1.1623,0.297788,-0.287870,0.80,0.001981,0.000776,0.003284,0.054434
2025-12-02,16.59,1.1596,0.297788,-0.287870,0.80,-0.002326,0.001981,0.000776,-0.037703
2025-12-03,16.08,1.1660,0.297788,-0.287870,0.80,0.005504,-0.002326,0.001981,-0.030741


In [37]:
df_model = df_clean.dropna()


In [38]:
df_model

,vix,fx,inflation_us_mom,inflation_eu_mom,spread_rates,log_return,ret_lag1,ret_lag2,vix_change
date,,,,,,,,,
1999-01-07,24.37,1.1672,0.182482,0.052356,0.53,0.003089,-0.010600,-0.004412,0.044130
1999-01-08,23.28,1.1554,0.182482,0.052356,0.78,-0.010161,0.003089,-0.010600,-0.044727
1999-01-11,25.46,1.1534,0.182482,0.052356,1.21,-0.001733,-0.010161,0.003089,0.093643
1999-01-12,28.10,1.1548,0.182482,0.052356,0.82,0.001213,-0.001733,-0.010161,0.103692
1999-01-13,30.11,1.1698,0.182482,0.052356,0.66,0.012906,0.001213,-0.001733,0.071530
...,...,...,...,...,...,...,...,...,...
2025-12-01,17.24,1.1623,0.297788,-0.287870,0.80,0.001981,0.000776,0.003284,0.054434
2025-12-02,16.59,1.1596,0.297788,-0.287870,0.80,-0.002326,0.001981,0.000776,-0.037703
2025-12-03,16.08,1.1660,0.297788,-0.287870,0.80,0.005504,-0.002326,0.001981,-0.030741


# Differentes approches de machine learning:

Dans cette partie, pour essayer de rester aligné avec notre objectif, qui est de prédire le taux de change juste en entrant la date, j'ai pris comme feature les varibales qu'on a dans df_model, et de predire les features de demain, donc predire tout ces varibles pour demain, donc on cherche à predire ce qu il se passe demain, et à la fin j ai codé une fonction, qui itère le modele de la derniere journée qu on a de df_model, pour remonter jusqu'à la date souhaité, donc on prédit jour par jour jusqu'à arriver à la date souhaité

## Modèle XGBOOST

In [39]:



targets = [
    'log_return',
    'vix',
    'inflation_us_mom',
    'inflation_eu_mom',
    'spread_rates'
]

features = [
    'ret_lag1',
    'ret_lag2',
    'vix_change',
    'vix',
    'inflation_us_mom',
    'inflation_eu_mom',
    'spread_rates'
]


df_sup = df_model.copy()


for col in targets:
    df_sup[f'y_{col}'] = df_sup[col].shift(-1)


y_cols = [f'y_{col}' for col in targets]
df_sup = df_sup[features + y_cols].dropna()


X = df_sup[features]
Y = df_sup[y_cols]


split = int(len(df_sup) * 0.8)
X_train, X_test = X.iloc[:split], X.iloc[split:]
Y_train, Y_test = Y.iloc[:split], Y.iloc[split:]

X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((5399, 7), (5399, 5), (1350, 7), (1350, 5))

In [40]:
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor


base = XGBRegressor(
    n_estimators=500,
    max_depth=3,
    learning_rate=0.03,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    random_state=42,
    n_jobs=-1
)

model = MultiOutputRegressor(base)


model.fit(X_train, Y_train)


Y_pred = model.predict(X_test)
Y_pred = pd.DataFrame(Y_pred, index=Y_test.index, columns=Y_test.columns)


rows = []
for col in Y_test.columns:
    y_true = Y_test[col].values
    y_hat = Y_pred[col].values
    rows.append({
        "target": col,
        "R2": r2_score(y_true, y_hat),
        "MAE": mean_absolute_error(y_true, y_hat),
        "RMSE": np.sqrt(mean_squared_error(y_true, y_hat)),
    })

metrics_df = pd.DataFrame(rows).sort_values("RMSE")
print(metrics_df)

#  Importance des features (par cible)
# (Chaque sortie a son propre estimateur XGBoost)
for i, col in enumerate(Y_test.columns):
    est = model.estimators_[i]
    imp = pd.Series(est.feature_importances_, index=X_train.columns).sort_values(ascending=False)
    print("\nFeature importance for", col)
    print(imp.head(10))

               target        R2       MAE      RMSE
0        y_log_return -0.012210  0.003509  0.004735
2  y_inflation_us_mom  0.923057  0.032720  0.074155
3  y_inflation_eu_mom  0.846597  0.073038  0.191672
4      y_spread_rates  0.956401  0.141018  0.204354
1               y_vix  0.888780  1.110715  1.831000

Feature importance for y_log_return
spread_rates        0.158325
vix                 0.145676
vix_change          0.143935
inflation_us_mom    0.142586
ret_lag1            0.138585
inflation_eu_mom    0.138391
ret_lag2            0.132502
dtype: float32

Feature importance for y_vix
vix                 0.773052
inflation_us_mom    0.147555
spread_rates        0.038794
inflation_eu_mom    0.021469
ret_lag1            0.006873
vix_change          0.006380
ret_lag2            0.005878
dtype: float32

Feature importance for y_inflation_us_mom
inflation_us_mom    0.803697
vix                 0.104175
spread_rates        0.038955
inflation_eu_mom    0.029523
vix_change          0.0082

In [41]:
def forecast_to_date(df_model, model, features, targets, target_date):

    df_future = df_model.copy()
    target_date = pd.to_datetime(target_date).normalize()

    current_date = df_future.index.max()

    while current_date < target_date:

        #  Récupérer les features à la date actuelle
        X_current = df_future.loc[[current_date], features]

        #  Prédire toutes les variables à t+1
        y_pred = model.predict(X_current)[0]

        #  Construire nouvelle ligne
        next_date = current_date + pd.Timedelta(days=1)

        new_row = {}
        for i, col in enumerate(targets):
            new_row[col] = y_pred[i]

        #  Recalcul des variables dérivées
        # fx via log_return
        fx_today = df_future.loc[current_date, 'fx']
        log_return_next = new_row['log_return']
        fx_next = fx_today * np.exp(log_return_next)

        new_row['fx'] = fx_next

        # ret_lags
        new_row['ret_lag2'] = df_future.loc[current_date, 'ret_lag1']
        new_row['ret_lag1'] = log_return_next

        # vix_change
        vix_today = df_future.loc[current_date, 'vix']
        vix_next = new_row['vix']
        new_row['vix_change'] = (vix_next - vix_today) / vix_today

        #  Ajouter au dataframe
        df_future.loc[next_date] = new_row

        current_date = next_date

    return df_future

In [42]:
taux_Xgboost = forecast_to_date(
    df_model=df_model,
    model=model,
    features=features,
    targets=['log_return','vix','inflation_us_mom','inflation_eu_mom','spread_rates'],
    target_date="2026-02-20"
)

In [43]:
taux_Xgboost

,vix,fx,inflation_us_mom,inflation_eu_mom,spread_rates,log_return,ret_lag1,ret_lag2,vix_change
date,,,,,,,,,
1999-01-07,24.370000,1.167200,0.182482,0.052356,0.530000,0.003089,-0.010600,-0.004412,0.044130
1999-01-08,23.280000,1.155400,0.182482,0.052356,0.780000,-0.010161,0.003089,-0.010600,-0.044727
1999-01-11,25.460000,1.153400,0.182482,0.052356,1.210000,-0.001733,-0.010161,0.003089,0.093643
1999-01-12,28.100000,1.154800,0.182482,0.052356,0.820000,0.001213,-0.001733,-0.010161,0.103692
1999-01-13,30.110000,1.169800,0.182482,0.052356,0.660000,0.012906,0.001213,-0.001733,0.071530
...,...,...,...,...,...,...,...,...,...
2026-02-16,15.674161,1.108620,0.258045,-0.111271,0.640952,-0.000691,-0.000691,-0.000691,0.000000
2026-02-17,15.674161,1.107854,0.258045,-0.111271,0.640952,-0.000691,-0.000691,-0.000691,0.000000
2026-02-18,15.674161,1.107088,0.258045,-0.111271,0.640952,-0.000691,-0.000691,-0.000691,0.000000


In [44]:
taux_Xgboost_valeur = taux_Xgboost["fx"].iloc[-1]
print("Predicted FX:", taux_Xgboost_valeur)

Predicted FX: 1.1055590518889529


## Régression linéaire (Méthode naïve):

In [45]:


# Modèle de base
base_model = LinearRegression()

# Multi-output
model_lr = MultiOutputRegressor(base_model)

# Entraînement
model_lr.fit(X_train, Y_train)

# Prédictions
Y_pred_lr = model_lr.predict(X_test)
Y_pred_lr = pd.DataFrame(Y_pred_lr, index=Y_test.index, columns=Y_test.columns)

In [46]:
rows = []

for col in Y_test.columns:
    y_true = Y_test[col].values
    y_hat = Y_pred_lr[col].values

    rows.append({
        "target": col,
        "R2": r2_score(y_true, y_hat),
        "MAE": mean_absolute_error(y_true, y_hat),
        "RMSE": np.sqrt(mean_squared_error(y_true, y_hat)),
    })

metrics_lr = pd.DataFrame(rows).sort_values("RMSE")
print(metrics_lr)

               target        R2       MAE      RMSE
0        y_log_return  0.003296  0.003497  0.004699
2  y_inflation_us_mom  0.948751  0.016407  0.060520
4      y_spread_rates  0.995032  0.018531  0.068983
3  y_inflation_eu_mom  0.938459  0.034914  0.121402
1               y_vix  0.888963  1.110485  1.829493


In [47]:
taux_Regre = forecast_to_date(df_model=df_model,
    model=model_lr,
    features=features,
    targets=['log_return','vix','inflation_us_mom','inflation_eu_mom','spread_rates'],
    target_date="2026-02-20")

In [48]:
taux_Regre

,vix,fx,inflation_us_mom,inflation_eu_mom,spread_rates,log_return,ret_lag1,ret_lag2,vix_change
date,,,,,,,,,
1999-01-07,24.370000,1.167200,0.182482,0.052356,0.530000,0.003089,-0.010600,-0.004412,0.044130
1999-01-08,23.280000,1.155400,0.182482,0.052356,0.780000,-0.010161,0.003089,-0.010600,-0.044727
1999-01-11,25.460000,1.153400,0.182482,0.052356,1.210000,-0.001733,-0.010161,0.003089,0.093643
1999-01-12,28.100000,1.154800,0.182482,0.052356,0.820000,0.001213,-0.001733,-0.010161,0.103692
1999-01-13,30.110000,1.169800,0.182482,0.052356,0.660000,0.012906,0.001213,-0.001733,0.071530
...,...,...,...,...,...,...,...,...,...
2026-02-16,17.791214,1.159583,0.212925,0.145387,0.616255,-0.000057,-0.000057,-0.000057,0.000825
2026-02-17,17.805688,1.159517,0.212364,0.145783,0.613175,-0.000057,-0.000057,-0.000057,0.000814
2026-02-18,17.819982,1.159450,0.211814,0.146153,0.610095,-0.000057,-0.000057,-0.000057,0.000803


In [49]:
taux_Regre_valeur = taux_Regre["fx"].iloc[-1]
print("Predicted FX:", taux_Regre_valeur)

Predicted FX: 1.1593173125651062


## Random forest

In [50]:
from sklearn.ensemble import RandomForestRegressor


# Modèle de base
base_rf = RandomForestRegressor(
    n_estimators=300,
    max_depth=6,
    random_state=42,
    n_jobs=-1
)

# Multi-output
model_rf = MultiOutputRegressor(base_rf)

# Entraînement
model_rf.fit(X_train, Y_train)

# Prédictions
Y_pred_rf = model_rf.predict(X_test)
Y_pred_rf = pd.DataFrame(Y_pred_rf, index=Y_test.index, columns=Y_test.columns)

In [51]:
rows = []

for col in Y_test.columns:
    y_true = Y_test[col].values
    y_hat = Y_pred_rf[col].values

    rows.append({
        "target": col,
        "R2": r2_score(y_true, y_hat),
        "MAE": mean_absolute_error(y_true, y_hat),
        "RMSE": np.sqrt(mean_squared_error(y_true, y_hat)),
    })

metrics_rf = pd.DataFrame(rows).sort_values("RMSE")
print(metrics_rf)

               target        R2       MAE      RMSE
0        y_log_return -0.002567  0.003508  0.004713
4      y_spread_rates  0.983866  0.074223  0.124313
2  y_inflation_us_mom  0.714053  0.064917  0.142955
3  y_inflation_eu_mom  0.825435  0.074668  0.204466
1               y_vix  0.885190  1.145114  1.860314


In [52]:
taux_rf = forecast_to_date(df_model=df_model,
    model=model_rf,
    features=features,
    targets=['log_return','vix','inflation_us_mom','inflation_eu_mom','spread_rates'],
    target_date="2026-02-20")

In [53]:
taux_rf

,vix,fx,inflation_us_mom,inflation_eu_mom,spread_rates,log_return,ret_lag1,ret_lag2,vix_change
date,,,,,,,,,
1999-01-07,24.370000,1.167200,0.182482,0.052356,0.530000,0.003089,-0.010600,-0.004412,4.413025e-02
1999-01-08,23.280000,1.155400,0.182482,0.052356,0.780000,-0.010161,0.003089,-0.010600,-4.472712e-02
1999-01-11,25.460000,1.153400,0.182482,0.052356,1.210000,-0.001733,-0.010161,0.003089,9.364261e-02
1999-01-12,28.100000,1.154800,0.182482,0.052356,0.820000,0.001213,-0.001733,-0.010161,1.036921e-01
1999-01-13,30.110000,1.169800,0.182482,0.052356,0.660000,0.012906,0.001213,-0.001733,7.153025e-02
...,...,...,...,...,...,...,...,...,...
2026-02-16,15.574733,1.172931,0.311799,-0.313239,0.686416,0.000111,0.000111,0.000111,2.281075e-16
2026-02-17,15.574733,1.173061,0.311799,-0.313239,0.686416,0.000111,0.000111,0.000111,-2.281075e-16
2026-02-18,15.574733,1.173191,0.311799,-0.313239,0.686416,0.000111,0.000111,0.000111,0.000000e+00


In [54]:
taux_rf_valeur = taux_rf["fx"].iloc[-1]
print("Predicted FX:", taux_rf_valeur)

Predicted FX: 1.1734517141111054


#### Une analyse des résultats:

Dans ce projet, plusieurs modèles ont été implémentés afin de prédire des variables financières journalières, notamment le rendement du taux de change EUR/USD (log_return), le VIX, les taux d’inflation et les spreads de taux d’intérêt. Les modèles testés sont :

Régression linéaire multi-sorties

Random Forest

XGBoost

**1. Rendement du taux de change (variable principale)**

La variable centrale de notre étude est le log-return journalier du taux de change EUR/USD. Les résultats obtenus montrent que :

La régression linéaire est légèrement plus performante que les autres modèles.

Random Forest obtient des performances proches.

XGBoost est légèrement en dessous pour cette variable.

Cependant, les différences de RMSE restent très faibles et les R² sont proches de zéro (voire légèrement négatifs). Cela indique que les rendements journaliers du taux de change sont très difficilement prédictibles.

Ce résultat est cohérent avec l’hypothèse de marche aléatoire et l’hypothèse d’efficience des marchés, selon lesquelles les prix financiers intègrent rapidement l’information disponible, rendant la prévision à court terme extrêmement complexe.

**2. Variables macroéconomiques (inflation et spread)**

Pour les variables évoluant plus lentement comme l’inflation et les spreads de taux, les modèles obtiennent des R² élevés.

La régression linéaire se révèle systématiquement la plus performante. Cela suggère que ces variables suivent une dynamique relativement stable et linéaire, ne nécessitant pas de modèles non linéaires complexes.

**3. VIX**

Les trois modèles obtiennent des performances similaires pour la prédiction du VIX. Cela s’explique par la forte autocorrélation de cet indice de volatilité, ce qui le rend plus facilement prévisible que les rendements de change.

**4. Interprétation des résultats**

Bien que cela puisse sembler contre-intuitif, le modèle le plus simple (régression linéaire) fournit les résultats les plus stables et légèrement supérieurs.

Les modèles complexes comme XGBoost et Random Forest n’apportent pas d’amélioration significative. Cela indique que :

Les variations journalières du taux de change contiennent une forte composante aléatoire.

Le signal prédictif est faible comparé au bruit de marché.

Ajouter de la complexité n’améliore pas la performance en absence de structure forte dans les données.

**5. Conclusion générale**

Les résultats empiriques confirment un principe fondamental en finance :

Les rendements journaliers des taux de change sont extrêmement difficiles à prédire, et même des modèles avancés de machine learning peinent à surpasser des approches simples.

Dans ce contexte, un modèle linéaire, plus stable et interprétable, apparaît comme le choix le plus pertinent.

On recherchera une le meilleur jour pour échanger, càd celui avec le plus petit taux d'échange:

In [55]:
def get_min_fx_from_last_train(df_model, df_projection):


    last_train_date = df_model.index.max()


    df_future = df_projection[df_projection.index > last_train_date]

    min_value = df_future['fx'].min()
    min_date = df_future['fx'].idxmin()

    return {
        "min_fx": float(min_value),
        "date": min_date
    }

Pour XGBOOST:

In [56]:
get_min_fx_from_last_train(df_model, taux_Xgboost)

{'min_fx': 1.1055590518889529, 'date': Timestamp('2026-02-20 00:00:00')}

Pour la régrssion

In [57]:
get_min_fx_from_last_train(df_model, taux_Regre)

{'min_fx': 1.1593173125651062, 'date': Timestamp('2026-02-20 00:00:00')}

In [58]:
get_min_fx_from_last_train(df_model, taux_rf)

{'min_fx': 1.1636282500871298, 'date': Timestamp('2025-12-06 00:00:00')}